In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
time_column = "Date"
window_size = 2

In [3]:
trading_df = pd.read_csv("datasets/EURUSD/15m.csv", nrows=2000).drop(["Volume"], axis=1)
trading_df["Date"] = pd.to_datetime(trading_df["Date"], dayfirst=True)
query_df = pd.read_csv("datasets/EURUSD/1h.csv", nrows=1000).drop(["Volume"], axis=1)
query_df["Date"] = pd.to_datetime(query_df["Date"], dayfirst=True)

In [6]:
trading_df.iloc[2].to_numpy()

array([Timestamp('2023-09-20 00:30:00'), 1.06836, 1.06859, 1.06825,
       1.06837], dtype=object)

In [4]:
trading_df.sample(100)

,Date,Open,High,Low,Close
828,2023-10-02 15:00:00,1.04956,1.05065,1.04956,1.05049
1332,2023-10-09 21:00:00,1.05679,1.05683,1.05650,1.05670
678,2023-09-29 01:30:00,1.05702,1.05728,1.05694,1.05708
1215,2023-10-06 15:45:00,1.05934,1.05996,1.05877,1.05949
82,2023-09-20 20:30:00,1.06614,1.06625,1.06598,1.06600
...,...,...,...,...,...
488,2023-09-27 02:00:00,1.05698,1.05699,1.05638,1.05639
126,2023-09-21 07:30:00,1.06445,1.06573,1.06394,1.06469
1971,2023-10-18 12:45:00,1.05559,1.05578,1.05493,1.05572
87,2023-09-20 21:45:00,1.06637,1.06654,1.06632,1.06649


In [30]:
stacked = pd.concat([query_df.drop([time_column], axis=1).add_suffix(f"_{shift}").shift(shift) for shift in range(window_size)], axis=1)
stacked = pd.concat([query_df[time_column], stacked], axis=1)
stacked.head()

,Date,Open_0,High_0,Low_0,Close_0,Open_1,High_1,Low_1,Close_1
0,2023-09-20 00:00:00,1.06817,1.06860,1.06795,1.06850,NaN,NaN,NaN,NaN
1,2023-09-20 01:00:00,1.06850,1.06877,1.06802,1.06864,1.06817,1.06860,1.06795,1.06850
2,2023-09-20 02:00:00,1.06865,1.06899,1.06829,1.06832,1.06850,1.06877,1.06802,1.06864
3,2023-09-20 03:00:00,1.06830,1.06844,1.06784,1.06797,1.06865,1.06899,1.06829,1.06832
4,2023-09-20 04:00:00,1.06797,1.06813,1.06784,1.06788,1.06830,1.06844,1.06784,1.06797


In [31]:
concat_trading = trading_df.copy()
concat_trading[time_column] = concat_trading[time_column].dt.floor(timedelta(hours=1))
concat_trading.head()

,Date,Open,High,Low,Close
0,2023-09-20 00:00:00,1.06817,1.06839,1.06795,1.06801
1,2023-09-20 00:00:00,1.06801,1.06838,1.06801,1.06835
2,2023-09-20 00:00:00,1.06836,1.06859,1.06825,1.06837
3,2023-09-20 00:00:00,1.06838,1.06860,1.06835,1.06850
4,2023-09-20 01:00:00,1.06850,1.06863,1.06834,1.06851


In [40]:
pd.merge(concat_trading[time_column], stacked, on=time_column, how='outer').head()

,Date,Open_0,High_0,Low_0,Close_0,Open_1,High_1,Low_1,Close_1
0,2023-09-20 00:00:00,1.06817,1.06860,1.06795,1.06850,NaN,NaN,NaN,NaN
1,2023-09-20 00:00:00,1.06817,1.06860,1.06795,1.06850,NaN,NaN,NaN,NaN
2,2023-09-20 00:00:00,1.06817,1.06860,1.06795,1.06850,NaN,NaN,NaN,NaN
3,2023-09-20 00:00:00,1.06817,1.06860,1.06795,1.06850,NaN,NaN,NaN,NaN
4,2023-09-20 01:00:00,1.06850,1.06877,1.06802,1.06864,1.06817,1.0686,1.06795,1.0685


In [44]:
stacked.iloc[0][["Close_0", "Close_1"]]

Close_0    1.0685
Close_1       NaN
Name: 0, dtype: object

In [15]:
df = pd.DataFrame()

In [37]:
# create two sample dataframes
df1 = pd.DataFrame({'date': ['2022-01-01', '2022-01-02', '2022-01-03'],
                    'value1': [1, 2, 3]})

df2 = pd.DataFrame({'date': ['2022-01-01', '2022-01-01', '2022-01-02', '2022-01-02', '2022-01-03'],
                    'value2': [4, 5, 6, 7, 8]})

# merge the two dataframes on the 'date' column
merged_df = pd.merge(df1, df2, on='date', how='outer')
merged_df.head()

,date,value1,value2
0,2022-01-01,1,4
1,2022-01-01,1,5
2,2022-01-02,2,6
3,2022-01-02,2,7
4,2022-01-03,3,8


In [2]:
import utils

ModuleNotFoundError: No module named 'utils'

In [11]:
from random import choice
ds = {"1": 1, "2": 2, "3": 3}
a, b = choice(list(ds.items()))
a, b

('3', 3)

In [15]:
max_length = 2

list(range(-max_length, max_length + 1))

[-2, -1, 0, 1, 2]